In [1]:
using DifferentialEquations, DiffEqProblemLibrary, ParameterizedFunctions, Plots, ODE, ODEInterfaceDiffEq
gr()

van = @ode_def_noinvhes VanDerPol2 begin
  dy = μ*((1-x^2)*y - x)
  dx = 1*y
end μ=>1.

prob = ODEProblem(VanDerPol2(μ=1e6),[0;2.],(0.0,6.3))
abstols = 1./10.^(6:9)
reltols = 1./10.^(3:6)

sol = solve(prob,CVODE_BDF(),abstol=1/10^14,reltol=1/10^14)
test_sol = TestSolution(sol.t,sol.u)

DiffEqDevTools.TestSolution{false,Array{Float64,1},Array{Array{Float64,1},1},Void}([0.0,2.64469e-17,2.80266e-14,1.15557e-13,2.03087e-13,2.90617e-13,3.78147e-13,5.62513e-13,8.97554e-13,1.48894e-12  …  6.29592,6.2964,6.29688,6.29737,6.29785,6.29834,6.29882,6.2993,6.29979,6.3],Array{Float64,1}[[0.0,2.0],[-5.28939e-11,2.0],[-5.60533e-8,2.0],[-2.31114e-7,2.0],[-4.06174e-7,2.0],[-5.81234e-7,2.0],[-7.56294e-7,2.0],[-1.12502e-6,2.0],[-1.79511e-6,2.0],[-2.97788e-6,2.0]  …  [1.38186,-1.42528],[1.38377,-1.42461],[1.38569,-1.42394],[1.38762,-1.42327],[1.38955,-1.42259],[1.3915,-1.42192],[1.39345,-1.42125],[1.39542,-1.42057],[1.39739,-1.4199],[1.39825,-1.4196]],nothing,false)

## Graphs 

These graphs are to show that the methods compute the correct solution.

In [2]:
sol = solve(prob,Rosenbrock23(),abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,denseplot=false,ylim=[-4;4])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 21.7148,186.806 21.7148,186.806 21.7148,186.81 21.7148,186.855 21.7148,186.904 21.7148,187.014 21.7148,187.141 21.7148,187.35 21.7148,187.586 21.7148,187.912 
 21.7148,188.273 21.7148,188.718 21.7148,189.199 21.7148,189.753 21.7148,190.34 21.7148,190.985 21.7148,191.659 21.7148,192.375 21.7148,193.114 21.7148,193.883 
 21.7148,194.667 21.7148,195.47 21.7148,196.281 21.7148,197.103 21.7148,197.926 21.7148,198.752 21.7148,199.575 21.7148,200.393 21.7148,201.203 21.7148,202.003 
 21.7148,202.791 21.7148,203.565 21.7148,204.323 21.7148,205.064 21.7148,205.786 21.7148,206.488 21.7148,207.169 21.7148,207.829 21.7148,208.465 21.7148,209.078 
 21.7148,209.667 21.7148,210.231 21.7148,210.771 21.7148,211.286 21.7148,211.776 21.7148,212.242 21.7148,212.682 21.7148,213.098 21.7148,213.49 21.7149,213.857 
 21.7149,214.202 21.7149,214.523 21.7149,214.821 21.7149,215.098 21.7149,215.353 21.7149,215.588 21.7149,215.803 21.7149,215.999 21.7149,216.177 21.7149,216.337 
 21.7149,216.481 21.7149,216.609 21.7149,216.722 21.7149,216.822 21.7149,216.909 21.7149,216.983 21.7149,217.047 21.7149,217.101 21.7149,217.145 21.715,217.181 
 21.715,217.21 21.715,217.233 21.715,217.25 21.715,217.263 21.715,217.272 21.715,217.277 21.7151,217.281 21.7151,217.283 21.7151,217.283 21.7152,217.284 
 21.7153,217.284 21.7155,217.284 21.7161,217.284 21.7181,217.284 21.7267,217.286 21.7562,217.291 21.7964,217.299 21.8733,217.313 21.9733,217.332 22.1207,217.359 
 22.2982,217.393 22.5252,217.435 22.7817,217.484 23.0826,217.541 23.4076,217.603 23.7689,217.673 24.1469,217.746 24.5528,217.826 24.9686,217.908 25.405,217.995 
 25.8458,218.084 26.3015,218.176 26.7574,218.27 27.2241,218.366 27.6881,218.463 28.1599,218.563 28.6271,218.662 29.0999,218.764 29.5669,218.866 30.0381,218.97 
 30.5027,219.073 30.9705,219.178 31.4312,219.283 31.8944,219.389 32.3503,219.495 32.8082,219.602 33.2588,219.709 33.7108,219.817 34.1556,219.924 34.6017,220.033 
 35.0406,220.142 35.4804,220.252 35.9133,220.361 36.3469,220.471 36.7737,220.581 37.2011,220.693 37.6217,220.804 38.0429,220.916 38.4576,221.027 38.8726,221.14 
 39.2813,221.252 39.6902,221.366 40.093,221.479 40.4959,221.593 40.8928,221.707 41.2898,221.822 41.6809,221.937 42.0721,222.053 42.4575,222.168 42.8429,222.285 
 43.2227,222.401 43.6023,222.518 43.9766,222.635 44.3507,222.753 44.7195,222.871 45.088,222.989 45.4515,223.108 45.8146,223.227 46.1727,223.347 46.5305,223.467 
 46.8834,223.587 47.2358,223.708 47.5836,223.829 47.9309,223.951 48.2736,224.073 48.6157,224.195 48.9534,224.318 49.2905,224.441 49.6233,224.564 49.9554,224.689 
 50.2833,224.813 50.6105,224.938 50.9337,225.063 51.2561,225.189 51.5745,225.314 51.8922,225.441 52.2059,225.568 52.5189,225.695 52.8281,225.822 53.1365,225.951 
 53.4412,226.079 53.745,226.208 54.0453,226.337 54.3446,226.467 54.6405,226.597 54.9355,226.728 55.227,226.859 55.5177,226.991 55.805,227.122 56.0913,227.255 
 56.3744,227.388 56.6566,227.521 56.9356,227.654 57.2136,227.789 57.4885,227.923 57.7625,228.058 58.0334,228.193 58.3033,228.33 58.5703,228.466 58.8362,228.603 
 59.0993,228.74 59.3614,228.878 59.6206,229.015 59.8788,229.154 60.1343,229.293 60.3887,229.433 60.6405,229.573 60.8912,229.713 61.1392,229.854 61.3863,229.995 
 61.6307,230.137 61.8741,230.279 62.115,230.422 62.3549,230.565 62.5923,230.709 62.8286,230.853 63.0626,230.998 63.2955,231.143 63.526,231.288 63.7555,231.435 
 63.9827,231.581 64.2088,231.728 64.4327,231.876 64.6555,232.024 64.8761,232.172 65.0957,232.321 65.3131,232.471 65.5294,232.621 65.7437,232.771 65.9569,232.922 
 66.168,233.074 66.3781,233.226 66.5862,233.378 66.7932,233.531 66.9982,233.685 67.2022,233.839 67.4043,233.993 67.6053,234.149 67.8044,234.304 68.0025,

In [3]:
sol = solve(prob,Rosenbrock32())
plot(sol,denseplot=false,ylim=[-4;4])

Rosenbrock32 does not do well on oscillatory problems since it's not L-stable!

In [8]:
sol = solve(prob,ode23s(),abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,ylim=[-4;4],denseplot=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 t 
 
 
 
 
 
 y(t) 
 
 
 
 x(t)

ODE.jl fails to solve the problem.

In [5]:
sol = solve(prob,CVODE_BDF(),abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,denseplot=false,ylim=[-4;4])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 21.7148,186.806 21.7148,186.806 21.7148,186.808 21.7148,186.816 21.7148,186.824 21.7148,186.833 21.7148,186.846 21.7148,186.869 21.7148,186.912 21.7148,187.003 
 21.7148,187.173 21.7148,187.342 21.7148,187.51 21.7148,187.677 21.7148,188.123 21.7148,188.562 21.7148,188.995 21.7148,189.421 21.7148,190.096 21.7148,191.127 
 21.7148,192.601 21.7148,193.991 21.7148,195.304 21.7148,196.542 21.7148,197.711 21.7148,198.814 21.7148,199.855 21.7148,201.516 21.7148,204.185 21.7148,206.403 
 21.7148,207.671 21.7148,208.791 21.7148,209.781 21.7148,211.081 21.7148,212.156 21.7148,213.045 21.7149,213.78 21.7149,214.387 21.7149,214.89 21.7149,215.305 
 21.7149,215.648 21.7149,215.931 21.7149,216.166 21.7149,216.359 21.7149,216.52 21.7149,216.652 21.7149,216.762 21.7149,216.895 21.7149,216.994 21.7149,217.068 
 21.7149,217.123 21.715,217.164 21.715,217.195 21.715,217.217 21.715,217.242 21.715,217.263 21.715,217.271 21.715,217.276 21.7151,217.279 21.7151,217.281 
 21.7151,217.282 21.7151,217.283 21.7151,217.283 21.7151,217.283 21.7151,217.284 21.7152,217.284 21.7152,217.284 21.7152,217.284 21.7152,217.284 21.7153,217.284 
 21.7153,217.284 21.7154,217.284 21.7154,217.284 21.7155,217.284 21.7156,217.284 21.7156,217.284 21.7158,217.284 21.716,217.284 21.7161,217.284 21.7165,217.284 
 21.7169,217.284 21.7172,217.284 21.7184,217.284 21.7205,217.285 21.7227,217.285 21.7442,217.289 21.8971,217.318 22.1294,217.361 22.3617,217.405 22.8363,217.494 
 23.311,217.585 23.7857,217.676 24.6954,217.854 26.2796,218.172 27.8637,218.5 29.4479,218.84 31.032,219.192 32.6162,219.557 34.2003,219.936 35.7845,220.329 
 37.7422,220.836 39.6999,221.369 41.6575,221.931 43.6152,222.523 45.5729,223.149 48.8177,224.269 50.8317,225.024 52.8457,225.831 54.8597,226.696 56.8737,227.626 
 58.8877,228.631 60.9017,229.721 62.9157,230.909 64.2019,231.726 65.488,232.595 66.7742,233.52 67.9331,234.408 69.0919,235.354 71.3551,237.39 72.4161,238.447 
 73.4771,239.577 74.538,240.792 75.599,242.103 76.66,243.522 77.7209,245.066 78.7819,246.755 79.8429,248.613 80.9038,250.671 81.4967,251.923 82.0896,253.257 
 82.6824,254.682 83.2753,256.211 83.8682,257.857 84.8662,260.936 85.3181,262.479 85.7701,264.129 86.222,265.901 86.6739,267.81 87.1259,269.874 87.5778,272.115 
 88.0298,274.563 88.4817,277.249 88.9337,280.215 89.1974,282.098 89.4611,284.107 89.7248,286.254 89.9885,288.561 90.2523,291.046 90.516,293.733 90.7797,296.654 
 91.0434,299.843 91.3071,303.345 91.5708,307.212 91.722,309.621 91.8732,312.19 92.102,316.418 92.2301,318.99 92.3583,321.73 92.4864,324.658 92.6145,327.793 
 92.7426,331.164 92.8708,334.801 92.9989,338.742 93.127,343.028 93.2551,347.719 93.3832,352.877 93.4583,356.151 93.5334,359.637 93.6084,363.351 93.6777,367.01 
 93.747,370.917 93.8561,377.641 93.9146,381.576 93.9731,385.776 94.0315,390.267 94.1036,396.252 94.1756,402.821 94.2476,410.079 94.3196,418.14 94.3698,424.318 
 94.42,431.04 94.4703,438.401 94.5205,446.479 94.5518,451.941 94.5832,457.778 94.6146,464.03 94.6459,470.745 94.6773,477.988 94.7086,485.831 94.74,494.357 
 94.7713,503.675 94.8027,513.919 94.834,525.249 94.8654,537.864 94.8968,552.035 94.9281,568.124 94.9595,586.59 94.9908,608.047 94.9941,610.541 94.9964,612.289 
 94.9987,614.043 95.001,615.815 95.0032,617.615 95.0055,619.44 95.0078,621.287 95.0101,623.157 95.0124,625.052 95.0146,626.972 95.0214,632.81 95.0281,638.89 
 95.0331,643.554 95.0381,648.371 95.0431,653.346 95.0481,658.485 95.0531,663.8 95.0581,669.303 95.0631,675.003 95.0712,684.698 95.0793,695.003 95.0874,705.972 
 95.0955,717.692 95.1036,730.281 95.1117,743.809 95.1175,754.141 95.1233,765.09 95.1276,773.556 95.1319,782.392 95.1362,791.638 95.1404,801.359 95.1447,811.555 
 95.1477,818.9

In [6]:
sol = solve(prob,radau(),abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,denseplot=false,ylim=[-4;4])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 21.7148,186.806 21.7148,187.706 21.7148,188.58 21.7148,192.144 21.7148,195.878 21.7148,200.094 21.7148,204.078 21.7148,207.139 21.7148,210.154 21.7148,212.272 
 21.7149,214.094 21.7149,215.253 21.7149,217.012 21.715,217.265 21.7151,217.283 21.7153,217.284 21.7159,217.284 21.7195,217.285 21.7486,217.29 21.8362,217.306 
 22.3434,217.401 24.6087,217.837 31.9371,219.399 45.6308,223.167 66.7946,233.533 80.2409,249.353 87.1637,270.031 90.9522,298.647 93.0031,338.712 94.1025,395.716 
 94.6845,478.469 94.9876,601.96 95.1201,749.466 95.189,936.962 95.2282,1189.18 95.2595,1786.72 95.2689,2289.66 95.275,2962.35 95.281,4517.48 95.2849,7354.57 
 95.2873,11912.4 95.2891,20254.2 95.2904,36074.8 95.2913,65535.5 95.292,120713 95.2925,224904 95.2929,422411 95.2931,798886 95.2933,1.52299e+06 95.2935,2.93634e+06 
 95.2936,5.77165e+06 95.2937,1.1834e+07 95.2938,2.16151e+07 95.2938,4.06291e+07 95.2938,5.57756e+07 95.2939,5.77851e+07 95.2939,3.16056e+07 95.294,1.02373e+07 95.294,1.80044e+06 95.2941,465937 
 95.2941,77989.5 95.2942,15668.6 95.2942,3272.79 95.2943,786.821 95.2944,181.745 95.2945,157.354 95.2945,156.408 95.2947,156.33 95.295,156.329 95.2956,156.329 
 95.2979,156.329 95.3028,156.328 95.3188,156.325 95.3857,156.312 95.6091,156.271 96.1708,156.165 97.1209,155.984 99.6529,155.483 104.556,154.434 116.092,151.429 
 141.471,139.247 154.515,122.904 160.245,105.904 164.289,77.928 166.547,36.3526 167.692,-22.0795 168.289,-108.466 168.591,-238.358 168.72,-396.191 168.786,-595.483 
 168.823,-877.791 168.852,-1518.87 168.861,-2098.62 168.866,-2861.81 168.871,-4581.08 168.875,-7753.81 168.877,-12967.8 168.879,-22520.9 168.88,-40600.4 168.881,-74344.8 
 168.882,-137643 168.882,-257252 168.882,-484229 168.883,-917586 168.883,-1.75322e+06 168.883,-3.39155e+06 168.883,-6.70846e+06 168.883,-1.39877e+07 168.883,-2.37943e+07 168.883,-3.93439e+07 
 168.883,-5.5335e+07 168.883,-5.73632e+07 168.883,-3.08556e+07 168.884,-1.03551e+07 168.884,-2.50471e+06 168.884,-776002 168.884,-146834 168.884,-34740.9 168.884,-8086.35 168.884,-123.49 
 168.884,203.298 168.884,216.708 168.884,217.258 168.884,217.282 168.885,217.282 168.886,217.282 168.891,217.283 168.91,217.287 169.025,217.308 169.414,217.381 
 170.318,217.552 171.795,217.838 174.945,218.479 180.052,219.617 189.969,222.268 209.912,230.795 225.097,245.363 233.575,266.6 237.869,295.58 240.112,336.517 
 241.273,394.977 241.874,481.041 242.179,610.749 242.309,768.025 242.375,966.913 242.413,1246.97 242.441,1882.83 242.45,2458.04 242.456,3216.14 242.461,4921.31 
 242.465,8065.71 242.467,13229.9 242.468,22689.6 242.47,40591.5 242.471,74000.9 242.471,136665 242.472,255068 242.472,479739 242.472,908640 242.472,1.73551e+06 
 242.473,3.35611e+06 242.473,6.63473e+06 242.473,1.38142e+07 242.473,2.35814e+07 242.473,3.94013e+07 242.473,5.53795e+07 242.473,5.73517e+07 242.473,3.08477e+07 242.473,1.03607e+07 242.473,2.50686e+06 
 242.473,777112 242.473,147315 242.473,35160.3 242.473,8475.64 242.473,497.782 242.473,170.343 242.474,156.904 242.474,156.353 242.474,156.329 242.474,156.329 
 242.475,156.329 242.48,156.328 242.499,156.324 242.615,156.303 243.003,156.23 243.906,156.059 245.383,155.773 248.533,155.132 253.641,153.994 263.56,151.342 
 283.507,142.813 298.689,128.245 307.162,107.021 311.457,78.0525 313.7,37.1416 314.861,-21.2799 315.463,-107.275 315.768,-236.862 315.898,-393.969 315.964,-592.586 
 316.002,-872.164 316.03,-1507.64 316.039,-2082.01 316.045,-2838.51 316.05,-4540.78 316.054,-7680.26 316.056,-12834.7 316.058,-22275.7 316.059,-40143.1 316.06,-73487.1 
 316.061,-136027 316.061,-254195 316.062,-478415 316.062,-906444 316.062,-1.73161e+06 316.062,-3.34875e+06 316.062,-6.61989e+06 316.062,-1.378e+07 316.062

In [4]:
setups = [Dict(:alg=>Rosenbrock23()),
          # Dict(:alg=>ode23s()), # fails
          Dict(:alg=>CVODE_BDF()),
          Dict(:alg=>rodas()),
          Dict(:alg=>radau())]
wp = ode_workprecision_set(prob,abstols,reltols,setups;dense=false,save_timeseries=false,appxsol=test_sol,maxiters=Int(1e5))
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.Rosenbrock23{0,true,Base.LinAlg.#lufact!} 
 
 
 
 Sundials.CVODE 
 
 
 BDF 
 
 
 :Newton,:Dense 
 
 
 
 ODEInterfaceDiffEq.rodas 
 
 
 
 ODEInterfaceDiffEq.radau

### Conclusion

Of the Julia-based methods, Rosenbrock23 is the better method. It is both faster and has less error. CVODE_BDF is faster, while radau solves with much lower error.